# Conditional Class Performance

Basic Modules:

In [1]:
import pandas
from pandas import DataFrame, Series
pandas.options.display.max_columns = None

import time
import copy

import matplotlib.pyplot as plt
%matplotlib inline

from itertools import combinations

#### Import Data File:
The data representation indicates student failing a class (1) or student passing a class (0)

In [2]:
df = pandas.read_excel('Class_Bundle_Sample_File.xlsx')

print(df.shape)

df.head()


(99, 11)


,EMPLID,CL1,CL2,CL3,CL4,CL5,CL6,CL7,CL8,CL9,CL10
0,1,1,0,0,0,0,1,1,0,0,0
1,2,0,1,1,1,1,1,0,0,0,1
2,3,1,0,0,0,0,1,1,0,0,1
3,4,0,1,0,1,1,1,0,1,1,0
4,5,0,1,0,1,0,0,0,1,1,0


##### Create all possible combinations of n classes

In [3]:
n = 4

classes = df.columns[1:]

comb = combinations(classes, n) 

comb_list = list(comb)

print('there are {} possible {}-class combinations'.format(len(comb_list), n))

there are 210 possible 4-class combinations


### Prepare data for processing

###### Fix the EMPLID column name which contains a space inserted at the end for no obvious reason.

In [4]:
df.columns

Index(['EMPLID ', 'CL1', 'CL2', 'CL3', 'CL4', 'CL5', 'CL6', 'CL7', 'CL8',
       'CL9', 'CL10'],
      dtype='object')

In [5]:
df.rename(columns={'EMPLID ':'EMPLID'},inplace=True)

## Engine

In [6]:
def combination_counter(df):
    
    #set up rows,columns,indexes and data to populate adjecency matrix
    data = []
    students = {}
    combo_dict = {}
    
    # iterate through all possible class-combos combos
    for combo in comb_list:
        # pull enrollment indicator for class-combos
        df_tuple = df[['EMPLID'] + list(combo)]
        # ignore class-combos without students in common
        if df_tuple[(df_tuple.iloc[:,1]==1) & 
                    (df_tuple.iloc[:,2]==1) & 
                    (df_tuple.iloc[:,3]==1) & 
                    (df_tuple.iloc[:,4]==1)].shape[0]!=0:
            
            # pull df of all students in common for a given class-combo
            common_stdts_emplid = df_tuple[(df_tuple.iloc[:,1]==1) & 
                                           (df_tuple.iloc[:,2]==1) & 
                                           (df_tuple.iloc[:,3]==1) & 
                                           (df_tuple.iloc[:,4]==1)]

            common_stdts = common_stdts_emplid.drop(columns=['EMPLID'])

            # count students in common for a given class-combo
            common_stdts_cnt = common_stdts.iloc[:,0].sum()

            # store students in common count
            data.append(common_stdts_cnt)

            # store students in common with emplid
            students[combo] = common_stdts_emplid

            # store index class-combos
            combo_dict[combo] = [combo[0],combo[1],combo[2],combo[3],common_stdts_cnt]
                
    return (combo_dict,students,data)

In [13]:
comb_list[0]

('CL1', 'CL2', 'CL3', 'CL4')

In [17]:
df_tuple = df[['EMPLID'] + list(comb_list[0])]
df_tuple[list(comb_list[0])].any()

CL1    True
CL2    True
CL3    True
CL4    True
dtype: bool

In [24]:
df_tuple.iloc[:,1:3]==1

,CL1,CL2
0,True,False
1,False,True
2,True,False
3,False,True
4,False,True
5,False,True
6,True,True
7,True,False
8,False,True
9,True,True


In [36]:
(df_tuple.iloc[:,1]==1) & (df_tuple.iloc[:,2]==1)

0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9      True
10     True
11     True
12     True
13    False
14     True
15    False
16    False
17    False
18     True
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28     True
29    False
      ...  
69    False
70    False
71    False
72     True
73    False
74    False
75    False
76     True
77    False
78     True
79    False
80    False
81    False
82    False
83    False
84     True
85     True
86    False
87     True
88     True
89    False
90    False
91     True
92    False
93    False
94    False
95    False
96     True
97    False
98    False
Length: 99, dtype: bool

In [39]:
df_tuple.iloc[:,1:3].all(axis=1)

0     False
1     False
2     False
3     False
4     False
5     False
6      True
7     False
8     False
9      True
10     True
11     True
12     True
13    False
14     True
15    False
16    False
17    False
18     True
19    False
20    False
21    False
22    False
23     True
24    False
25    False
26    False
27    False
28     True
29    False
      ...  
69    False
70    False
71    False
72     True
73    False
74    False
75    False
76     True
77    False
78     True
79    False
80    False
81    False
82    False
83    False
84     True
85     True
86    False
87     True
88     True
89    False
90    False
91     True
92    False
93    False
94    False
95    False
96     True
97    False
98    False
Length: 99, dtype: bool

In [38]:
df_tuple.iloc[:,1:3]

,CL1,CL2
0,1,0
1,0,1
2,1,0
3,0,1
4,0,1
5,0,1
6,1,1
7,1,0
8,0,1
9,1,1


###### Implement Engine and obtain results

In [7]:
start_time = time.time()
combo_dict, students, data = combination_counter(df)
end_time = time.time()

print('run took {:0.3f} seconds'.format((end_time-start_time)))

run took 0.764 seconds


In [8]:
print('there are {} class-tuples that have at least one student failing all n classes'.format(len(data)))

print('the maximum number of students enrolled in a class set is {}'.format(max(data)))

there are 207 class-tuples that have at least one student failing all n classes
the maximum number of students enrolled in a class set is 14


In [9]:
combo_data = pandas.DataFrame.from_dict(combo_dict, orient='index')
combo_data.columns = ['class1','class2','class3','class4','failed']

combo_data.sort_values(by='failed',ascending=False,inplace=True)

combo_data.head()

,class1,class2,class3,class4,failed
"(CL2, CL5, CL6, CL7)",CL2,CL5,CL6,CL7,14
"(CL2, CL3, CL5, CL6)",CL2,CL3,CL5,CL6,14
"(CL1, CL2, CL7, CL9)",CL1,CL2,CL7,CL9,13
"(CL1, CL3, CL7, CL9)",CL1,CL3,CL7,CL9,13
"(CL2, CL3, CL7, CL9)",CL2,CL3,CL7,CL9,12


###### Validation:
Look at original dataset and manually count events

In [10]:
df[['CL2','CL5','CL6','CL7']].sort_values(by=['CL2','CL5','CL6','CL7'],ascending=False).head(15)

,CL2,CL5,CL6,CL7
9,1,1,1,1
18,1,1,1,1
28,1,1,1,1
31,1,1,1,1
33,1,1,1,1
36,1,1,1,1
37,1,1,1,1
57,1,1,1,1
59,1,1,1,1
68,1,1,1,1
